# DATA STRUCT BLOCK
Đoạn code trên thực hiện các thao tác phân tích dữ liệu điểm thi của các thí sinh theo từng khối và xác định khối thi tốt nhất cho từng thí sinh, dựa trên các môn thi của họ.

In [ ]:
import pandas as pd
import requests
from io import StringIO
import os
from datetime import datetime, timedelta

# Load the data
data_api = 'https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download'
tinh_api = 'https://andyanh.id.vn/index.php/s/zbHTAjksBekNB4M/download'

def fetch_csv_from_api(api_url):
    
    cache_file = 'data_cache.csv' if 'p7XMy' in api_url else 'tinh_cache.csv'
    cache_timeout = timedelta(hours=24)
    
    if os.path.exists(cache_file):
        modified_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - modified_time < cache_timeout:
            print(f"Đang tải dữ liệu từ cache {cache_file}...")
            return pd.read_csv(cache_file)
    
    print(f"Đang tải dữ liệu từ API {api_url}...")
    response = requests.get(api_url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv(cache_file, index=False)
        return df
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Tải dữ liệu từ API
try:
    data_df = fetch_csv_from_api(data_api)
    tinh_df = fetch_csv_from_api(tinh_api)
    print("Đã tải dữ liệu thành công từ API")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu từ API: {e}")
    print("Không thể tải dữ liệu. Vui lòng kiểm tra kết nối internet và thử lại.")
    exit()

# Merge data based on 'MaTinh'
merged_df = pd.merge(data_df, tinh_df, on='MaTinh', how='left')

# Define block calculations
merged_df['Khoi_A'] = merged_df[['Toan', 'Ly', 'Hoa']].mean(axis=1)
merged_df['Khoi_B'] = merged_df[['Toan', 'Sinh', 'Hoa']].mean(axis=1)
merged_df['Khoi_C'] = merged_df[['Van', 'Lich su', 'Dia ly']].mean(axis=1)
merged_df['Khoi_D'] = merged_df[['Toan', 'Van', 'Ngoai ngu']].mean(axis=1)

# Check if any block has exactly two scores (indicating "Thí Sinh Thi Lại")
merged_df['Thi_Lai'] = (
    (merged_df[['Toan', 'Ly', 'Hoa']].notnull().sum(axis=1) == 2) |
    (merged_df[['Toan', 'Sinh', 'Hoa']].notnull().sum(axis=1) == 2) |
    (merged_df[['Van', 'Lich su', 'Dia ly']].notnull().sum(axis=1) == 2) |
    (merged_df[['Toan', 'Van', 'Ngoai ngu']].notnull().sum(axis=1) == 2)
)

# Determine the best block for students not marked as "Thi Lai"
merged_df['Best_Khoi'] = merged_df[['Khoi_A', 'Khoi_B', 'Khoi_C', 'Khoi_D']].idxmax(axis=1)
merged_df['Best_Khoi'] = merged_df.apply(lambda row: "Thí Sinh Thi Lại" if row['Thi_Lai'] else row['Best_Khoi'], axis=1)

# Select relevant columns for output
block_analysis_final_df = merged_df[['SBD', 'Best_Khoi', 'TenTinh']]

# Export the final result to CSV
block_analysis_final_df.to_csv('output/Block_Analysis_Final.csv', index=False)
